![image](images/um_logo.png)

# Computación II


# ***Sockets IV***

## Sockets UDP
Los sockets UDP (User Datagram Protocol) representan un aspecto fundamental en el vasto mundo de las comunicaciones de red. A diferencia de los sockets TCP, que se centran en la transmisión fiable de datos, los sockets UDP abordan una perspectiva más liviana y eficiente, brindando una forma rápida y directa de enviar información en entornos donde la velocidad y la simplicidad son prioritarias sobre la entrega garantizada.

En contraste con el flujo continuo de datos ofrecido por TCP, los sockets UDP utilizan datagramas individuales, unidades de información independientes que contienen los datos enviados junto con metadatos como la dirección de origen y destino. Esta abordaje de "tirar y olvidar" significa que no hay confirmaciones ni control de flujo incorporados en el protocolo UDP, lo que puede llevar a una entrega no confiable y a la posibilidad de que los datagramas lleguen en un orden diferente al que se enviaron.

A pesar de estas limitaciones, los sockets UDP son extremadamente útiles en ciertos contextos. Son adecuados para aplicaciones donde la velocidad es crítica, como transmisiones de video en tiempo real, juegos en línea o servicios de voz. Además, debido a su menor sobrecarga, son preferibles en situaciones donde se prioriza la eficiencia por encima de la integridad completa de los datos.

El uso de sockets UDP requiere una mayor responsabilidad por parte del desarrollador en términos de manejo de errores, detección de pérdida de paquetes y control del orden de los datagramas. Sin embargo, este enfoque ofrece la flexibilidad de ajustar el equilibrio entre velocidad y confiabilidad según las necesidades específicas de la aplicación.

## Programando un servidor

In [ ]:
#!/usr/bin/python3
import socket

# create a socket object
serversocket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) 

serversocket.bind(('0.0.0.0', 2323))

data, add = serversocket.recvfrom(1024)

serversocket.sendto(b'Hola cliente', add)



## Cliente netcat

En el contexto de las comunicaciones mediante sockets UDP, se presenta la posibilidad de que varios clientes envíen mensajes a un mismo servidor de manera simultánea. Cada cliente puede crear su propio socket UDP y establecer una conexión independiente con el servidor utilizando la dirección IP y el número de puerto correspondientes. A medida que los clientes envían mensajes a través de sus respectivos sockets, el servidor, que también opera mediante sockets UDP, puede recibir estos mensajes de manera independiente y sin necesidad de mantener un estado persistente de conexión. Esta arquitectura permite una comunicación eficiente y paralela entre múltiples clientes y un servidor central, lo que resulta especialmente útil en situaciones donde se necesita procesar solicitudes de manera rápida y sin sobrecargar la red.

## Programando un cliente

In [ ]:
#!/usr/bin/python3
import socket

clientsocket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
#clientsocket.bind(('localhost', 0))
clientsocket.sendto(b'Hola server', ('localhost', 2323))

## El módulo socketserver

El módulo socketserver de Python ofrece una manera conveniente y eficiente de crear servidores de red. Al proporcionar una abstracción de alto nivel sobre los sockets, este módulo simplifica el proceso de implementar servidores que escuchan y responden a conexiones de clientes. Con socketserver, es posible desarrollar aplicaciones de servidor sin tener que lidiar directamente con la complejidad de la creación y gestión de sockets individuales. Además, este módulo permite la creación de servidores tanto TCP como UDP, y ofrece una variedad de clases base que puedes extender para personalizar el comportamiento del servidor según tus necesidades. 

https://docs.python.org/es/3.10/library/socketserver.html

### Esquema de herencia de las clases base

![image](images/socket_server.png)

### MixIn

![image](images/socket_server_mixin.png)

### Programando un servidor no concurrente

In [ ]:
import socketserver

class MyUDPHandler(socketserver.BaseRequestHandler):
    """
    This class works similar to the TCP handler class, except that
    self.request consists of a pair of data and client socket, and since
    there is no connection the client address must be given explicitly
    when sending data back via sendto().
    """

    def handle(self):
        data = self.request[0].strip()
        socket = self.request[1]
        print("{} wrote:".format(self.client_address[0]))
        print(data)
        socket.sendto(data.upper(), self.client_address)

if __name__ == "__main__":
    HOST, PORT = "localhost", 2323
    with socketserver.UDPServer((HOST, PORT), MyUDPHandler) as server:
        server.serve_forever()

### Programando un servidor concurrente (con cliente de prueba en el mismo código)

In [ ]:
import socket
import threading
import socketserver

class ThreadedTCPRequestHandler(socketserver.BaseRequestHandler):

    def handle(self):
        data = str(self.request.recv(1024), 'ascii')
        cur_thread = threading.current_thread()
        response = bytes("{}: {}".format(cur_thread.name, data), 'ascii')
        self.request.sendall(response)

class ThreadedTCPServer(socketserver.ThreadingMixIn, socketserver.TCPServer):
    pass

def client(ip, port, message):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.connect((ip, port))
        sock.sendall(bytes(message, 'ascii'))
        response = str(sock.recv(1024), 'ascii')
        print("Received: {}".format(response))

if __name__ == "__main__":
    # Port 0 means to select an arbitrary unused port
    HOST, PORT = "localhost", 0

    server = ThreadedTCPServer((HOST, PORT), ThreadedTCPRequestHandler)
    with server:
        ip, port = server.server_address

        # Start a thread with the server -- that thread will then start one
        # more thread for each request
        server_thread = threading.Thread(target=server.serve_forever)
        # Exit the server thread when the main thread terminates
        server_thread.daemon = True
        server_thread.start()
        print("Server loop running in thread:", server_thread.name)

        client(ip, port, "Hello World 1")
        client(ip, port, "Hello World 2")
        client(ip, port, "Hello World 3")

        server.shutdown()